# Memory

In [29]:
from dotenv import load_dotenv
load_dotenv()

True

In [30]:
from langchain_core.chat_history import BaseChatMessageHistory


class InMemoryHistory(BaseChatMessageHistory):
    def __init__(self):
        super().__init__()
        self.messages = []
    
    def add_messages(self, messages):
        self.messages.extend(messages)
        
    def clear(self):
        self.messages = []
        
    def __repr__(self):
        return f'InMemoryHistory(messages={str(self.messages)})'

In [31]:
# item(key=session_id, value=InMemotyHisotry인스턴스)
store = {}

def get_by_session_id(session_id):
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return store[session_id]

In [32]:
from langchain_core.messages import HumanMessage

history = get_by_session_id('user1')
history.add_message(HumanMessage(content='점심 뭐 먹지?'))
print(store)

{'user1': InMemoryHistory(messages=[HumanMessage(content='점심 뭐 먹지?', additional_kwargs={}, response_metadata={})])}


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate

"""
prompt = ChatPromptTemplate.from_messages([
    ('system', '너는 {skill}을 잘하는 AI 어시스턴트야'),
    MessagesPlaceholder(variable_name='history'),
    ('human', '{query}')
]) 
"""

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template('너는 {skill}을 잘하는 AI 어시스턴트야'),
    MessagesPlaceholder(variable_name='history'),
    HumanMessagePromptTemplate('{query}')
])
model = ChatOpenAI(model='gpt-4o-mini')

chain = prompt | model

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_by_session_id,
    input_messages_key='query',
    history_messages_key='history'
)

In [34]:
response  =chain_with_history.invoke(
    {'skill': '대화', 'query': '다람쥐는 도초리 나무 세 그루를 키우고 있다.'},
    config={'configurable': {'session_id': 'squirrel'}}
)

print(response)

content='다람쥐가 도토리 나무를 키우고 있다니 정말 귀엽고 흥미로운 설정이네요! 도토리 나무는 다람쥐에게 중요한 식량 공급원이기도 하죠. 어떤 이야기를 생각하고 계신가요? 다람쥐가 나무를 어떻게 돌보는지, 나무에서 자란 도토리를 어떻게 사용하는지에 대한 이야기도 좋을 것 같아요!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 97, 'prompt_tokens': 45, 'total_tokens': 142, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CBwbVxklFkpc5kPyn9pJaWXCoLf11', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--348d8fea-ba9b-45a7-84ce-c57bb51590d6-0' usage_metadata={'input_tokens': 45, 'output_tokens': 97, 'total_tokens': 142, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [35]:
store

{'user1': InMemoryHistory(messages=[HumanMessage(content='점심 뭐 먹지?', additional_kwargs={}, response_metadata={})]),
 'squirrel': InMemoryHistory(messages=[HumanMessage(content='다람쥐는 도초리 나무 세 그루를 키우고 있다.', additional_kwargs={}, response_metadata={}), AIMessage(content='다람쥐가 도토리 나무를 키우고 있다니 정말 귀엽고 흥미로운 설정이네요! 도토리 나무는 다람쥐에게 중요한 식량 공급원이기도 하죠. 어떤 이야기를 생각하고 계신가요? 다람쥐가 나무를 어떻게 돌보는지, 나무에서 자란 도토리를 어떻게 사용하는지에 대한 이야기도 좋을 것 같아요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 97, 'prompt_tokens': 45, 'total_tokens': 142, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CBwbVxklFkpc5kPyn9pJaWXCoLf11', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--348d8fea-ba9b-45a7-84ce-c

In [37]:
response = chain_with_history.invoke(
    {'skill': '대화', 'query': '다람쥐는 도토리 나무를 몇 그루 가지고 있지?'},
    config={'configurable': {'session_id': 'squirrel'}}
)

In [38]:
print(response)

content='다람쥐는 도토리 나무를 세 그루 가지고 있어요! 도토리 나무는 다람쥐에게 소중한 식량을 제공하고, 그늘을 만들어 주기도 하죠. 이 나무들로 인해 다람쥐는 더 안전하고 풍요로운 생활을 할 수 있을 것 같습니다. 더 궁금한 점이 있으신가요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 86, 'prompt_tokens': 296, 'total_tokens': 382, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CBwcsXAhhhhgSOvDpJEyOsKxbYJeF', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--a8931f2f-fd3a-457e-a8bd-869196ca0440-0' usage_metadata={'input_tokens': 296, 'output_tokens': 86, 'total_tokens': 382, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [39]:
store

{'user1': InMemoryHistory(messages=[HumanMessage(content='점심 뭐 먹지?', additional_kwargs={}, response_metadata={})]),
 'squirrel': InMemoryHistory(messages=[HumanMessage(content='다람쥐는 도초리 나무 세 그루를 키우고 있다.', additional_kwargs={}, response_metadata={}), AIMessage(content='다람쥐가 도토리 나무를 키우고 있다니 정말 귀엽고 흥미로운 설정이네요! 도토리 나무는 다람쥐에게 중요한 식량 공급원이기도 하죠. 어떤 이야기를 생각하고 계신가요? 다람쥐가 나무를 어떻게 돌보는지, 나무에서 자란 도토리를 어떻게 사용하는지에 대한 이야기도 좋을 것 같아요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 97, 'prompt_tokens': 45, 'total_tokens': 142, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8bda4d3a2c', 'id': 'chatcmpl-CBwbVxklFkpc5kPyn9pJaWXCoLf11', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--348d8fea-ba9b-45a7-84ce-c